In [ ]:
CREATE OR REPLACE NETWORK RULE external_network_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('en.wikipedia.org:443');
  
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION wikipedia_access_integration
  ALLOWED_NETWORK_RULES = (wikipedia_network_rule)
  ENABLED = true;


CREATE OR REPLACE FUNCTION wiki_via_python(url STRING)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.8
HANDLER = 'get_page'
EXTERNAL_ACCESS_INTEGRATIONS = (wikipedia_access_integration)
PACKAGES = ('requests', 'beautifulsoup4')
AS
$$
import _snowflake
import requests
from bs4 import BeautifulSoup

def get_page(url):
  response = requests.get(f"{url}")
  soup = BeautifulSoup(response.text)
  return soup.get_text()
$$;

In [ ]:
-- Extract a Single URL
select wiki_via_python('https://en.wikipedia.org/wiki/67th_Annual_Grammy_Awards');

In [ ]:
-- Cortex Summarize has a 32,000 token limit
-- https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions
with api_result as (
    select left(wiki_via_python('https://en.wikipedia.org/wiki/Permian%E2%80%93Triassic_extinction_event'), 32000) as res
)
select SNOWFLAKE.CORTEX.SUMMARIZE(res) from api_result;

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
link = st.text_input(label='Input Your Wikipedia Link to Summarize the Page')

summary = '' 

if link:
    with st.spinner('Fetching Summary...'):
        session = get_active_session()
        df = session.sql(f"with api_result as (select left(wiki_via_python('{link}'), 32000) as res) select SNOWFLAKE.CORTEX.SUMMARIZE(res) from api_result")
        summary = df.collect()[0][0]
        st.write(summary)

In [ ]:
summary = '''The 67th Annual Grammy Awards honored the best recordings, compositions, and artists from September 16, 2023, to August 30, 2024. The ceremony took place at Crypto.com Arena in Los Angeles on February 2, 2025, and was broadcast on CBS and Paramount+. Kendrick Lamar's "Not Like Us" won five awards, including Record and Song of the Year, making it the most decorated rap song in Grammy history. Beyoncé received the most nominations with eleven and won three awards, including Album of the Year for Cowboy Carter. Chappell Roan and Sabrina Carpenter were among the other notable winners. The ceremony carried a renewed sense of purpose, raising additional funds for wildfire relief efforts and honoring first responders.'''
print(summary.replace("\'", "\\'"))

In [ ]:
set summary = 'The 67th Annual Grammy Awards honored the best recordings, compositions, and artists from September 16, 2023, to August 30, 2024. The ceremony took place at Crypto.com Arena in Los Angeles on February 2, 2025, and was broadcast on CBS and Paramount+. Kendrick Lamar\'s "Not Like Us" won five awards, including Record and Song of the Year, making it the most decorated rap song in Grammy history. Beyoncé received the most nominations with eleven and won three awards, including Album of the Year for Cowboy Carter. Chappell Roan and Sabrina Carpenter were among the other notable winners. The ceremony carried a renewed sense of purpose, raising additional funds for wildfire relief efforts and honoring first responders.'

In [ ]:
st.write(summary)

html_format_btn = st.button('Generate Markdown for Summary')

if html_format_btn:
    with st.spinner('Generating Markdown...'):
        summary_formatted = summary.replace("\'", "\\'")
        prompt = 'Generate a beautifully formatted slack message using slack markdown formatting, using the following text, but do not include additional messages before or after the resopnse. Where there should be a new line, use a new line operator instead.';
        summary_with_markdown = session.sql(f"select snowflake.cortex.complete('llama3.1-405b',concat('{prompt}','{summary_formatted}'));").collect()[0][0]
        st.markdown(f'{summary_with_markdown}')

In [ ]:
CREATE OR REPLACE NETWORK RULE slack_webhook_network_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('hooks.slack.com');

CREATE OR REPLACE SECRET my_slack_webhook_secret
  TYPE = GENERIC_STRING
  SECRET_STRING = '';

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION slack_webhook_access_integration
  ALLOWED_NETWORK_RULES = (slack_webhook_network_rule)
  ALLOWED_AUTHENTICATION_SECRETS = (my_slack_webhook_secret)
  ENABLED = true;

In [ ]:
CREATE OR REPLACE PROCEDURE send_slack_message(MSG string)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.10
HANDLER = 'main'
EXTERNAL_ACCESS_INTEGRATIONS = (slack_webhook_access_integration)
SECRETS = ('slack_url' = my_slack_webhook_secret)
PACKAGES = ('snowflake-snowpark-python', 'requests')
EXECUTE AS CALLER
AS
$$
import snowflake.snowpark as snowpark
import json
import requests
import _snowflake
from datetime import date

def main(session, msg):
    webhook_url = _snowflake.get_generic_secret_string('slack_url')

    slack_data = {"message": f"{msg}"}

    response = requests.post(
        webhook_url, data=json.dumps(slack_data),
        headers={'Content-Type': 'application/json'}
    )
    if response.status_code != 200:
        raise ValueError(
            'Request to slack returned an error %s, the response is:\n%s'
            % (response.status_code, response.text)
        )
    
    return "SUCCESS"
$$;

In [ ]:
st.write(f'{summary_with_markdown}')

send_slack_btn = st.button('Send Summary to Slack')   

if send_slack_btn:
    with st.spinner('Sending summary to Slack...'):
        summary_markdown_formatted = summary_with_markdown.replace("\'", "\\'")
        result = session.sql(f"call send_slack_message('{summary_markdown_formatted}')").collect()
        st.write(result)